In [612]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

### Reading samples

In [302]:
volfor = pd.read_csv('sample/volume_forecast.csv')
volfor.head()

,Agency,SKU,Volume
0,Agency_23,SKU_06,0
1,Agency_34,SKU_20,0
2,Agency_24,SKU_02,0
3,Agency_40,SKU_06,0
4,Agency_42,SKU_32,0


In [303]:
volfor.shape

(1450, 3)

In [304]:
print(volfor['Agency'].unique())
print(len(volfor['Agency'].unique()))

['Agency_23' 'Agency_34' 'Agency_24' 'Agency_40' 'Agency_42' 'Agency_08'
 'Agency_35' 'Agency_13' 'Agency_07' 'Agency_56' 'Agency_11' 'Agency_21'
 'Agency_18' 'Agency_50' 'Agency_60' 'Agency_17' 'Agency_29' 'Agency_26'
 'Agency_01' 'Agency_33' 'Agency_02' 'Agency_47' 'Agency_41' 'Agency_03'
 'Agency_36' 'Agency_10' 'Agency_31' 'Agency_37' 'Agency_44' 'Agency_16'
 'Agency_49' 'Agency_32' 'Agency_54' 'Agency_38' 'Agency_48' 'Agency_25'
 'Agency_55' 'Agency_46' 'Agency_59' 'Agency_58' 'Agency_20' 'Agency_57'
 'Agency_12' 'Agency_30' 'Agency_45' 'Agency_39' 'Agency_04' 'Agency_09'
 'Agency_43' 'Agency_53' 'Agency_27' 'Agency_28' 'Agency_19' 'Agency_05'
 'Agency_51' 'Agency_22' 'Agency_52' 'Agency_15']
58


In [305]:
print(volfor['SKU'].unique())
print(len(volfor['SKU'].unique()))

['SKU_06' 'SKU_20' 'SKU_02' 'SKU_32' 'SKU_26' 'SKU_18' 'SKU_27' 'SKU_04'
 'SKU_12' 'SKU_14' 'SKU_28' 'SKU_08' 'SKU_21' 'SKU_24' 'SKU_22' 'SKU_34'
 'SKU_07' 'SKU_05' 'SKU_03' 'SKU_23' 'SKU_15' 'SKU_01' 'SKU_31' 'SKU_11'
 'SKU_17']
25


In [306]:
skureco = pd.read_csv('sample/sku_recommendation.csv')
skureco.head()

,Agency,SKU
0,Agency_06,SKU_11
1,Agency_06,SKU_11
2,Agency_14,SKU_11
3,Agency_14,SKU_11


In [307]:
skureco.shape

(4, 2)

### Reading test

In [308]:
volfortst = pd.read_csv('test/volume_forecast.csv')
volfortst.head()

,Agency,SKU,Volume
0,Agency_23,SKU_06,NaN
1,Agency_34,SKU_20,NaN
2,Agency_24,SKU_02,NaN
3,Agency_40,SKU_06,NaN
4,Agency_42,SKU_32,NaN


In [309]:
skurecotst = pd.read_csv('sample/sku_recommendation.csv')
skurecotst.head()

,Agency,SKU
0,Agency_06,SKU_11
1,Agency_06,SKU_11
2,Agency_14,SKU_11
3,Agency_14,SKU_11


### Reading train

In [310]:
psp = pd.read_csv('train/price_sales_promotion.csv')
psp.head()

,Agency,SKU,YearMonth,Price,Sales,Promotions
0,Agency_01,SKU_01,201301,1141.500000,1033.432731,108.067269
1,Agency_01,SKU_01,201302,1141.500000,1065.417195,76.082805
2,Agency_01,SKU_01,201303,1179.345820,1101.133633,78.212187
3,Agency_01,SKU_01,201304,1226.687500,1138.283357,88.404143
4,Agency_01,SKU_01,201305,1230.331104,1148.969634,81.361470


In [311]:
psp.shape

(21000, 6)

In [312]:
psp.isnull().sum()

Agency        0
SKU           0
YearMonth     0
Price         0
Sales         0
Promotions    0
dtype: int64

In [313]:
print(len(psp['Agency'].unique()))
print(len(psp['SKU'].unique()))

58
25


In [314]:
histvol = pd.read_csv('train/historical_volume.csv')
histvol.head()

,Agency,SKU,YearMonth,Volume
0,Agency_22,SKU_01,201301,52.2720
1,Agency_22,SKU_02,201301,110.7000
2,Agency_58,SKU_23,201301,0.0000
3,Agency_48,SKU_07,201301,28.3200
4,Agency_22,SKU_05,201301,238.5387


In [315]:
histvol.shape

(21000, 4)

In [316]:
print(len(histvol['Agency'].unique()))
print(len(histvol['SKU'].unique()))

58
25


In [317]:
weather = pd.read_csv('train/weather.csv')
weather.head()

,YearMonth,Agency,Avg_Max_Temp
0,201301,Agency_01,17.072
1,201302,Agency_01,19.984
2,201303,Agency_01,24.600
3,201304,Agency_01,27.532
4,201305,Agency_01,29.396


In [318]:
weather.shape

(3600, 3)

In [319]:
iss = pd.read_csv('train/industry_soda_sales.csv')
iss.head()

,YearMonth,Soda_Volume
0,201301,718394219
1,201302,753938444
2,201303,892192092
3,201304,838099501
4,201305,864420003


In [320]:
iss.shape

(60, 2)

In [321]:
events = pd.read_csv('train/event_calendar.csv')
events.head()

,YearMonth,Easter Day,Good Friday,New Year,Christmas,Labor Day,Independence Day,Revolution Day Memorial,Regional Games,FIFA U-17 World Cup,Football Gold Cup,Beer Capital,Music Fest
0,201301,0,0,1,0,0,0,0,0,0,0,0,0
1,201302,0,0,0,0,0,0,0,0,0,0,0,0
2,201303,1,1,0,0,0,0,0,0,0,0,0,1
3,201304,0,0,0,0,0,0,0,0,0,0,0,0
4,201305,0,0,0,0,1,0,0,0,0,0,0,0


In [322]:
events.shape

(61, 13)

In [323]:
indvol = pd.read_csv('train/industry_volume.csv')
indvol.head()

,YearMonth,Industry_Volume
0,201301,492612703
1,201302,431937346
2,201303,509281531
3,201304,532390389
4,201305,551755254


In [324]:
indvol.shape

(60, 2)

In [325]:
demog = pd.read_csv('train/demographics.csv')
demog.head()

,Agency,Avg_Population_2017,Avg_Yearly_Household_Income_2017
0,Agency_05,3044268,182944
1,Agency_02,3137874,240809
2,Agency_55,2025793,238553
3,Agency_53,1729177,111048
4,Agency_50,2057678,144594


In [326]:
demog.shape

(60, 3)

### Combining dataset

In [327]:
train = psp.copy()
train.head()

,Agency,SKU,YearMonth,Price,Sales,Promotions
0,Agency_01,SKU_01,201301,1141.500000,1033.432731,108.067269
1,Agency_01,SKU_01,201302,1141.500000,1065.417195,76.082805
2,Agency_01,SKU_01,201303,1179.345820,1101.133633,78.212187
3,Agency_01,SKU_01,201304,1226.687500,1138.283357,88.404143
4,Agency_01,SKU_01,201305,1230.331104,1148.969634,81.361470


In [328]:
train = pd.merge(train, histvol, on=['Agency','SKU','YearMonth'])

In [329]:
train = pd.merge(train, weather, on=['Agency','YearMonth'])

In [330]:
train = pd.merge(train, iss, on=['YearMonth'])

In [331]:
train = pd.merge(train, events, on=['YearMonth'])

In [332]:
train = pd.merge(train, indvol, on=['YearMonth'])

In [333]:
train = pd.merge(train, demog, on=['Agency'])

In [334]:
train.head()

,Agency,SKU,YearMonth,Price,Sales,Promotions,Volume,Avg_Max_Temp,Soda_Volume,Easter Day,...,Independence Day,Revolution Day Memorial,Regional Games,FIFA U-17 World Cup,Football Gold Cup,Beer Capital,Music Fest,Industry_Volume,Avg_Population_2017,Avg_Yearly_Household_Income_2017
0,Agency_01,SKU_01,201301,1141.500000,1033.432731,108.067269,80.6760,17.072,718394219,0,...,0,0,0,0,0,0,0,492612703,153733,120207
1,Agency_01,SKU_02,201301,1074.157799,969.186209,104.971591,78.4080,17.072,718394219,0,...,0,0,0,0,0,0,0,492612703,153733,120207
2,Agency_01,SKU_03,201301,1239.734043,1141.894806,97.839237,35.9550,17.072,718394219,0,...,0,0,0,0,0,0,0,492612703,153733,120207
3,Agency_01,SKU_04,201301,1725.711827,1467.681860,258.029967,230.2956,17.072,718394219,0,...,0,0,0,0,0,0,0,492612703,153733,120207
4,Agency_01,SKU_05,201301,1260.322817,1172.763381,87.559436,15.9750,17.072,718394219,0,...,0,0,0,0,0,0,0,492612703,153733,120207


In [335]:
train.columns

Index(['Agency', 'SKU', 'YearMonth', 'Price', 'Sales', 'Promotions', 'Volume',
       'Avg_Max_Temp', 'Soda_Volume', 'Easter Day', 'Good Friday', 'New Year',
       'Christmas', 'Labor Day', 'Independence Day', 'Revolution Day Memorial',
       'Regional Games ', 'FIFA U-17 World Cup', 'Football Gold Cup',
       'Beer Capital', 'Music Fest', 'Industry_Volume', 'Avg_Population_2017',
       'Avg_Yearly_Household_Income_2017'],
      dtype='object')

In [336]:
train['Year'] = 0
train['Month'] = 0

In [337]:
for i,row in train.iterrows():
    train.at[i,'Year'] = int(str(row['YearMonth'])[:4])
    train.at[i,'Month'] = int(str(row['YearMonth'])[4:])

In [338]:
train['Year'].replace([2013, 2014, 2015, 2016, 2017],[0,1,2,3,4],inplace=True)

### Testing model on validaton set

In [339]:
model = train.copy()
model.head()

,Agency,SKU,YearMonth,Price,Sales,Promotions,Volume,Avg_Max_Temp,Soda_Volume,Easter Day,...,Regional Games,FIFA U-17 World Cup,Football Gold Cup,Beer Capital,Music Fest,Industry_Volume,Avg_Population_2017,Avg_Yearly_Household_Income_2017,Year,Month
0,Agency_01,SKU_01,201301,1141.500000,1033.432731,108.067269,80.6760,17.072,718394219,0,...,0,0,0,0,0,492612703,153733,120207,0,1
1,Agency_01,SKU_02,201301,1074.157799,969.186209,104.971591,78.4080,17.072,718394219,0,...,0,0,0,0,0,492612703,153733,120207,0,1
2,Agency_01,SKU_03,201301,1239.734043,1141.894806,97.839237,35.9550,17.072,718394219,0,...,0,0,0,0,0,492612703,153733,120207,0,1
3,Agency_01,SKU_04,201301,1725.711827,1467.681860,258.029967,230.2956,17.072,718394219,0,...,0,0,0,0,0,492612703,153733,120207,0,1
4,Agency_01,SKU_05,201301,1260.322817,1172.763381,87.559436,15.9750,17.072,718394219,0,...,0,0,0,0,0,492612703,153733,120207,0,1


In [340]:
X_data1 = model[model['Year'] < 4]
X_data2 = model[model['Year'] == 4]

In [341]:
X_data2 = X_data2[X_data2['Month']==1]

In [342]:
agencies = volfortst['Agency'].unique().tolist()
skus = volfortst['SKU'].unique().tolist()

In [343]:
xcols = ['Price', 'Sales', 'Promotions',
       'Avg_Max_Temp', 'Soda_Volume', 'Easter Day', 'Good Friday', 'New Year',
       'Christmas', 'Labor Day', 'Independence Day', 'Revolution Day Memorial',
       'Regional Games ', 'FIFA U-17 World Cup', 'Football Gold Cup',
       'Beer Capital', 'Music Fest', 'Industry_Volume', 'Avg_Population_2017',
       'Avg_Yearly_Household_Income_2017', 'Year', 'Month']

In [344]:
rf_grid = {
    'n_estimators': [3,4,5,7,10],
    'min_samples_leaf': [3,5,10,15,20],
    'n_jobs': [-1],
    'verbose': [1]
}
rfr = RandomForestRegressor()

In [345]:
def relative_error(y_true, y_pred):
    num = 0
    den = 0
    for t, p in zip(y_true, y_pred):
        num += abs(t - p)
        den += t
    return (1 - (num /(den + 1)))

In [351]:
scores = list()
for a in agencies:
    for s in skus:
        X_tr1 = X_data1[(X_data1['Agency']==a)&(X_data1['SKU']==s)]
        X_tr,y_tr = X_tr1[xcols], X_tr1['Volume']
        X_tr2 = X_data2[(X_data2['Agency']==a)&(X_data2['SKU']==s)]
        X_val,y_val = X_tr2[xcols], X_tr2['Volume']
        if (X_tr1.shape[0] > 0) and (X_tr2.shape[0] > 0):
            temp = list()
            for val in ParameterGrid(rf_grid):        
                rfr.set_params(**val)
                rfr.fit(X_tr,y_tr)
                y_pred = rfr.predict(X_val)
                error = relative_error(y_val,y_pred)
                temp.append(error)
            scores.append(temp)

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s fi

...

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s fi

In [365]:
index = list()
for i,val in enumerate(scores):
    maxval = max(val)
    for j,val1 in enumerate(val):
        if val1 == maxval:
            index.append(j) 

### Creating test dataset

In [522]:
test = volfortst.copy()
test.head()

,Agency,SKU,Volume
0,Agency_23,SKU_06,NaN
1,Agency_34,SKU_20,NaN
2,Agency_24,SKU_02,NaN
3,Agency_40,SKU_06,NaN
4,Agency_42,SKU_32,NaN


In [524]:
model_temp = model[(model['Month']==12) & (model['Year']==4)]

In [525]:
test = pd.merge(test,model_temp[['Agency', 'SKU', 'YearMonth', 'Price', 'Sales', 'Promotions','Avg_Max_Temp', 'Soda_Volume', 'Industry_Volume', 'Avg_Population_2017','Avg_Yearly_Household_Income_2017']], on = ['Agency','SKU'], how='outer')

In [528]:
model_temp1 = model[(model['Month']==1) & (model['Year']==4)]

In [529]:
test['Year'] = 5
test['Month'] = 1

In [530]:
test = pd.merge(test,model_temp1[['Agency', 'SKU','Easter Day', 'Good Friday', 'New Year','Christmas', 'Labor Day', 'Independence Day', 'Revolution Day Memorial','Regional Games ', 'FIFA U-17 World Cup', 'Football Gold Cup','Beer Capital', 'Music Fest' ]], on = ['Agency','SKU'], how='outer')

In [533]:
params_list = list()
for val in ParameterGrid(rf_grid):
    params_list.append(val)

In [534]:
index = list()
for i,val1 in enumerate(scores):
    maxval = max(val1)
    for j,val2 in enumerate(val1):
        if val2 == maxval:
            index.append(j)

In [541]:
test.fillna(0,inplace=True)

In [543]:
test['New Year'] = 1

In [547]:
val = 0
for a in agencies:
    for s in skus:
        X_tr1 = train[(train['Agency']==a)&(train['SKU']==s)]
        X_tr,y_tr = X_tr1[xcols], X_tr1['Volume']
        X_tr2 = test[(test['Agency']==a)&(test['SKU']==s)]
        X_val,y_val = X_tr2[xcols], X_tr2['Volume']
        if (X_tr1.shape[0] > 0) and (X_tr2.shape[0] > 0):
            params = params_list[index[val]]
            rfr.set_params(**params)
            rfr.fit(X_tr,y_tr)
            y_pred = rfr.predict(X_val)
            val += 1
            test.at[int(X_val.index.values),'Volume'] = float(y_pred)

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 |

[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 |

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:

[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 |

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s fi

[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   7 out of   7 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 o

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 |

In [548]:
test[['Agency','SKU','Volume']].to_csv('subrf.csv',index=False)

### SKU Recommendation

In [491]:
histvol[['SKU','Volume']].groupby(['SKU']).sum()

,Volume
SKU,
SKU_01,9.965440e+06
SKU_02,6.890135e+06
SKU_03,6.312349e+06
SKU_04,4.135040e+06
SKU_05,3.928825e+06
SKU_06,3.066250e+04
SKU_07,2.783793e+04
SKU_08,2.810187e+04
SKU_11,1.933691e+03


In [492]:
for i,row in skurecotst.iterrows():
    if (i+2) % 2 == 0:
        skurecotst.at[i,'SKU'] = 'SKU_01'
    else:
        skurecotst.at[i,'SKU'] = 'SKU_02'

In [493]:
skurecotst.to_csv('sub2max.csv', index=False)  

### Ensembling

In [595]:
model_temp.head()

,Agency,SKU,YearMonth,Price,Sales,Promotions,Volume,Avg_Max_Temp,Soda_Volume,Easter Day,...,Regional Games,FIFA U-17 World Cup,Football Gold Cup,Beer Capital,Music Fest,Industry_Volume,Avg_Population_2017,Avg_Yearly_Household_Income_2017,Year,Month
318,Agency_01,SKU_01,201712,1663.797492,1390.112272,273.685220,35.5320,19.403655,919709619,0,...,0,0,0,0,0,618073219,153733,120207,4,12
319,Agency_01,SKU_02,201712,1697.575472,1412.680031,284.895441,17.1720,19.403655,919709619,0,...,0,0,0,0,0,618073219,153733,120207,4,12
320,Agency_01,SKU_03,201712,1640.111950,1443.480950,196.631000,10.9134,19.403655,919709619,0,...,0,0,0,0,0,618073219,153733,120207,4,12
321,Agency_01,SKU_04,201712,2562.440869,2093.641230,468.799639,80.7057,19.403655,919709619,0,...,0,0,0,0,0,618073219,153733,120207,4,12
322,Agency_01,SKU_05,201712,2025.049474,1572.014619,453.034855,15.1443,19.403655,919709619,0,...,0,0,0,0,0,618073219,153733,120207,4,12


In [596]:
model2 = volfortst.copy()
model2.head()

,Agency,SKU,Volume
0,Agency_23,SKU_06,NaN
1,Agency_34,SKU_20,NaN
2,Agency_24,SKU_02,NaN
3,Agency_40,SKU_06,NaN
4,Agency_42,SKU_32,NaN


In [597]:
model2['Volume'] = 0

In [598]:
model2 = pd.merge(model2,model_temp[['Agency', 'SKU','Volume']], on = ['Agency','SKU'], how='outer')

In [599]:
model2 = model2.rename(columns={'Volume_y': 'Volume'})

In [600]:
model2.drop('Volume_x',axis=1,inplace=True)

In [601]:
model2.fillna(0,inplace=True)

In [602]:
model2.to_csv('subdec.csv',index=False)

In [603]:
model2['avg'] = 0

In [604]:
ens = test[['Agency','SKU','Volume']]
ens.head()

,Agency,SKU,Volume
0,Agency_23,SKU_06,0.0000
1,Agency_34,SKU_20,0.0000
2,Agency_24,SKU_02,119.2836
3,Agency_40,SKU_06,0.0000
4,Agency_42,SKU_32,0.0000


In [605]:
model2['avg'] = 0.6 * model2['Volume'] + 0.4 * ens['Volume']

In [606]:
model2.drop('Volume',axis=1,inplace=True)

In [607]:
model2 = model2.rename(columns={'avg': 'Volume'})

In [608]:
model2.to_csv('avgrfensemblev2.csv',index=False)

### Linear Model

In [614]:
lr = LinearRegression()
for a in agencies:
    for s in skus:
        X_tr1 = train[(train['Agency']==a)&(train['SKU']==s)]
        X_tr,y_tr = X_tr1[xcols], X_tr1['Volume']
        X_tr2 = test[(test['Agency']==a)&(test['SKU']==s)]
        X_val,y_val = X_tr2[xcols], X_tr2['Volume']
        if (X_tr1.shape[0] > 0) and (X_tr2.shape[0] > 0):
            lr.fit(X_tr,y_tr)
            y_pred = lr.predict(X_val)
            test.at[int(X_val.index.values),'Volume'] = float(y_pred)

In [615]:
test.head()

,Agency,SKU,Volume,YearMonth,Price,Sales,Promotions,Avg_Max_Temp,Soda_Volume,Industry_Volume,...,New Year,Christmas,Labor Day,Independence Day,Revolution Day Memorial,Regional Games,FIFA U-17 World Cup,Football Gold Cup,Beer Capital,Music Fest
0,Agency_23,SKU_06,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agency_34,SKU_20,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Agency_24,SKU_02,81.368404,201712.0,1719.482444,1401.710815,317.771629,26.91561,919709619.0,618073219.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Agency_40,SKU_06,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Agency_42,SKU_32,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
